# Notebook para testear herramientas

Combinar carta de observacion con fchart

Astroplan para planear segun altura y masa de aire

reportlab para generar el pdf de observacion

In [ ]:
import fchart
import os

import math as m
import numpy as np
import matplotlib.pyplot as plt
import seaborn

from astropy.coordinates import SkyCoord
from astropy.coordinates import get_sun
from astropy.time import Time
from astropy import units as u
from astropy.table import Table
from astropy.table import Column
from astropy.io import ascii
from astroplan import FixedTarget

import conf as cf

%matplotlib inline

In [ ]:
from astropy.utils.console import ProgressBar

In [ ]:
plots = cf.plots
if not os.path.isdir(plots):
    os.mkdir(plots)

In [ ]:
observatory = cf.observer

sunset_tonight = observatory.sun_set_time(cf.obs_time, which='next', horizon=-15*u.degree)
sunrise_tonight = observatory.sun_rise_time(cf.obs_time, which='next', horizon=-15*u.degree)

In [ ]:
print sunrise_tonight.sidereal_time('mean', longitude=cf.longitude).hms
print sunset_tonight.sidereal_time('mean', longitude=cf.longitude).hms

In [ ]:
def is_up_tonight(alpha, delta, sunrise, sunset, observatory):
    lat = observatory.location.latitude.degree
    circum = 90 - abs(lat)
    if abs(delta) > circum and np.sign(delta)==np.sign(lat):
        return True
    lon = observatory.location.longitude.degree
    lst_rise = sunrise.sidereal_time('mean', longitude=lon).degree
    lst_set = sunset.sidereal_time('mean', longitude=lon).degree
    
    if alpha%360 > (lst_set - 70)%360 and alpha%360 < (lst_rise + 70)%360:
        return True
    return False

In [ ]:
def alpha_cuts(observation_time, horizon=-15*u.degree, min_height=45*u.degree):
    sun = get_sun(observation_time)
    horiz = horizon.to(u.degree)
    h = min_height.to(u.degree)
    
    lowest_alpha = (sun.ra - horiz + h)
    highest_alpha = (sun.ra + horiz - h)#.hourangle%24

    return (lowest_alpha.to(u.hourangle).value, highest_alpha.to(u.hourangle).value)

In [ ]:
low, high = alpha_cuts(cf.obs_time, horizon=-15*u.degree, min_height=40*u.degree)

In [ ]:
print low, high

In [ ]:
observatory.local_sidereal_time(Time.now(), 'mean')

In [ ]:
white_cat= cf.catalog

white_table = ascii.read(white_cat, delimiter=' ', format='commented_header')

circum_angle = abs(90.*u.degree - abs(observatory.location.latitude))

circum = abs(white_table['Dec']*u.degree - 90.*u.degree) > circum_angle + 90*u.degree

alpha_obs_min, alpha_obs_max = alpha_cuts(cf.obs_time)

dist_lim = 80.

In [ ]:
print alpha_obs_max*u.hourangle, alpha_obs_min*u.hourangle

In [ ]:
near = white_table['Dist'] < dist_lim 
visible = white_table['App_Mag']< 19     # Apparent Magnitude cut
bright = white_table['Abs_Mag']< -17.5      # Absolute Magnitude cut
lim_dec = white_table['Dec']< 30. 

if alpha_obs_max > alpha_obs_min:
    alfa_min = white_table['RA'] >  alpha_obs_min      # Alpha cut
    alfa_max = white_table['RA'] <= alpha_obs_max

    sample = white_table[near & visible & bright & lim_dec & (alfa_min & alfa_max) | circum]
else:
    print "REVERSAL"
    alfa_min1 = white_table['RA'] > alpha_obs_min 
    alfa_min2 = white_table['RA'] < 24
    alfa_max1 = white_table['RA'] < alpha_obs_max 
    alfa_max2 = white_table['RA'] >= 0
    
    alfa_min = alfa_min1 & alfa_min2
    alfa_max = alfa_max1 & alfa_max2
    
    sample = white_table[near & visible & bright & lim_dec & (alfa_min | alfa_max) | circum]
    
sample

In [ ]:
plt.hist(sample['App_Mag'])
plt.xlabel('App B Mag')
plt.ylabel('Number')
plt.title('App B Mag sample histogram')

plt.savefig(os.path.join(plots, 'appmag_sample_histogram.png'), dpi=300)

plt.show()

In [ ]:
plt.hist(sample['Abs_Mag'])

In [ ]:
plt.hist(sample['RA'], bins=24)
plt.xlim((0,24))
plt.xlabel('RA [h]')
plt.ylabel('Number')
plt.title('Right Ascension sample histogram')

plt.savefig(os.path.join(plots, 'RA_sample_histogram.png'), dpi=300)

plt.show()

In [ ]:
plt.hist(sample['Dist'], range=[1,dist_lim])
plt.title('Distance histogram of the objects\n observable from Macon')

plt.xlabel('Distance [Mpc]')
plt.ylabel('Number')

plt.savefig(os.path.join(plots, 'distance_histogram_sample.png'), dpi=300)

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(211, projection="aitoff")
deg2rad=np.pi/180.

coord = SkyCoord(ra=sample['RA']*u.hourangle, dec=sample['Dec']*u.degree, frame='icrs')

mean_zenith_ra = np.mean(coord.ra.wrap_at(360*u.degree).radian)
if mean_zenith_ra > m.pi:
    mean_zenith_ra -= 2*m.pi
zenith_dec = observatory.location.latitude.radian

xg = coord.ra.wrap_at(180 * u.deg).radian
yg = coord.dec.radian

# we should avoid the galactic plane
plt.plot(xg,yg, "r.")
plt.plot(mean_zenith_ra, zenith_dec, 'bo')
plt.grid(True)
plt.title("Aitoff de objetos observables EABA")
plt.xlabel("Right Ascention [deg]")
plt.ylabel("Declination [deg]")
plt.savefig(os.path.join(plots, 'radec_aitoff_sample.png'), dpi=300)
plt.show()

In [ ]:
targets = []
rises = np.empty(len(sample), dtype='str')
sets = np.empty(len(sample), dtype='str')
i = 0
with ProgressBar(len(sample), ipython_widget=True) as bar:
    for row in sample:
        coordinates = SkyCoord(row['RA']*u.degree, row['Dec']*u.degree, frame='icrs')
        obj = FixedTarget(coord=coordinates, name=row['Name'])
        #targets.append(obj)
        rise_time = observatory.target_rise_time(cf.obs_time, obj)
        set_time = observatory.target_set_time(cf.obs_time, obj)
        if rise_time.jd != -999.0: rises[i] = rise_time.iso
        if set_time.jd != -999.0: sets[i] = set_time.iso
        i += 1
        bar.update()

In [ ]:
targcol = Column(targets, name='planTargets')
risecol = Column(rises, name='RiseTime')
setcol = Column(sets, name='SetTime')

sample.add_columns(targcol)
sample.add_columns(risecol)
sample.add_columns(setcol)